In [89]:
import time 
import re
import requests
import numpy as np
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.service import Service  
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [90]:
# start by defining the options 
options = Options()
options.headless = True
options.add_argument('--log-level=3')
driver = webdriver.Chrome(options = options)

C:\Users\shubhangib\AppData\Local\Temp\ipykernel_804\2608793959.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [91]:
# Requesting to get the url
url = "https://www.canfinhomes.com/Branch-locator.aspx" 
driver.get(url)
time.sleep(3)

In [59]:
# Clicking on View All button to access data 
WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.NAME,"ctl00$ContentPlaceHolder1$Button3"))).click()
time.sleep(10)

In [60]:
# Scraping using BeautifulSoup
html_text = driver.page_source
soup = BeautifulSoup(html_text, 'html.parser')

In [61]:
# Empty list to collect all dataframes from within the loop
dfs = []

In [70]:
# Preventing data from getting truncated
pd.set_option('display.max_colwidth', None)

# Main table has nested tables, seperate for each branch information
for table in soup.find_all('table', id= 'ctl00_ContentPlaceHolder1_dg_branch'):
    # Accessing all branch tables one by one
    for subtable in table.find_all('table', {'class' : 'tabcontent1'}):
        # Readinf and converting table dimensions to make it compatible to comvert into a dataframe
        info = pd.read_html(str(subtable))
        info = np.array(info)
        df = pd.DataFrame(info.tolist(), columns = ["Name"])
        df['Name'] = df['Name'].astype('str')
        # Seperating columns into address and remaining details
        df2 = df.replace(", '",'_', regex=True)
        df2 = pd.DataFrame(df2)
        df2[['Address', 'Details']] = df2.Name.str.split("_", expand = True)
        # Seperating column with remaining details into telephone and email
        df2[['Telephone', 'Email']] = df2.Details.str.split("Email ID:", expand = True)
        # Cleaning and dropping unnecessary columns
        df2['Address'] = df2['Address'].str[2:-1]
        df2['Email'] = df2['Email'].str[:-12]
        df3 = df2.drop(["Name", "Details"], axis = 1)
        df3['sample'] = df3['Address'].replace(" ",'', regex=True)
        df3['pincode'] = re.findall('\d+', str(df3['sample']))[-1]
        df4 = df3.drop(["sample"], axis = 1)
#         print(df4)
        #Appending all dataframes one into one list
        dfs.append(df4)

In [ ]:
# Concatenating all dataframes one below the other to create final dataset
appended_data = pd.concat(dfs).drop_duplicates().reset_index(drop=True)
# print(appended_data)
appended_data.to_excel("page1_results.xlsx", index = False)

######This code only collects results from first page. Working on getting remaining pages.

In [100]:
# for table in soup.find_all('table', id= 'ctl00_ContentPlaceHolder1_dg_branch'):
#     for tr in table.find_all('tr', {'class' : 'vergridfooter'}):
#         for td in tr.find_all('td'):
#             for a in td.find_all('a'):
#                 print(a)
#                 WebDriverWait(driver,15).until(EC.element_to_be_clickable((By.TAG_NAME,"a"))).click()
# #                 time.sleep(15)
# #                 pd.set_option('display.max_colwidth', None)
# #                 for table in soup.find_all('table', id= 'ctl00_ContentPlaceHolder1_dg_branch'):
# #                     for subtable in table.find_all('table', {'class' : 'tabcontent1'}):
# #                         info = pd.read_html(str(subtable))
# #                         info = np.array(info)
# #                         df = pd.DataFrame(info.tolist(), columns = ["Name"])
# #                         df['Name'] = df['Name'].astype('str')
# #                         df2 = df.replace(", '",'_', regex=True)
# #                         df2 = pd.DataFrame(df2)
# #                         df2[['Address', 'Details']] = df2.Name.str.split("_", expand = True)
# #                         df2[['Telephone', 'Email']] = df2.Details.str.split("Email ID:", expand = True)
# #                         df2['Address'] = df2['Address'].str[2:-1]
# #                         df2['Email'] = df2['Email'].str[:-12]
# #                         df3 = df2.drop(["Name", "Details"], axis = 1)
# #                         df3['sample'] = df3['Address'].replace(" ",'', regex=True)
# #                         df3['pincode'] = re.findall('\d+', str(df3['sample']))[-1]
# #                         df4 = df3.drop(["sample"], axis = 1)
# #                         print(df4)
# #                         time.sleep(5)